In [14]:
# Read in data to dataframe
import pandas as pd
from pandas import read_pickle
import os


In [15]:
filenames = os.listdir(r'/Users/apostoloskalatzis/Downloads/ML project/acute_stress/no_overlap/normalized')

li = []
for filename in filenames:
    dataframe=pd.read_csv(os.path.join(r'/Users/apostoloskalatzis/Downloads/ML project/acute_stress/no_overlap/normalized',filename))
    li.append(dataframe)

data = pd.concat(li, axis=0, ignore_index=True)

In [16]:
columns = [ 'HR_mean',  'HRV_RMSSD','HRV_MeanNN', 'HRV_LFHF', 'RSP_mean', 'RRV_RMSSD', 'RRV_MeanBB','label','ID']

In [17]:
acute_stress = data[columns]

In [18]:
# Read in data to dataframe
import pandas as pd
from pandas import read_pickle

names = ["acute_stress"]
dataframes = acute_stress

In [19]:
# GAN parameters
from sklearn.utils.fixes import loguniform
from sklearn.model_selection import ParameterSampler

param_dist = {
    "log_frequency": [True, False],
    "generator_lr": loguniform(0.00002, 0.002),
    "generator_decay": loguniform(0.00002, 0.002),
    "discriminator_decay": loguniform(0.00002, 0.002),
    "discriminator_lr": loguniform(0.00002, 0.002),
    "discriminator_decay": loguniform(0.00002, 0.002),
    "discriminator_steps": [1, 5, 10],
}

In [20]:
# function for RandomParameterSearch, given a data set name, dataframe, and parameters to search over.
from sdv.evaluation import evaluate
from sdv.tabular import CTGAN
from time import time


def RandomSearch(df: pd.DataFrame, params: dict, n_iter: int, random_seed=15
) -> dict:

    # generate random sets of parameters sampled over parameter space
    gan_parameter_sampler = ParameterSampler(
        param_distributions=params, n_iter=n_iter, random_state=random_seed
    )
    random_search_param_list = list(gan_parameter_sampler)

    best_parameter_set = {}
    best_score = 0
    count = 1
    # append empty parameter list for default values
    random_search_param_list.append({})

    # loop over all parameter sets and evaluate performance.
    ########################################################
    for parameter_set in random_search_param_list:
        print(
            f'Parameter Search for "{acute_stress}" dataset. Iteration {count} of {N_ITER+1}'
        )
        # print("Parameters:")
        # print("-------------------------------------")
        # for x in parameter_set:
        #     print(x, ": ", parameter_set[x])
        # if not parameter_set:
        #     print("Default parameters.")
        # print("-------------------------------------")
        start_time = time()
        score = 0
        if parameter_set:
            # if the parameter set is not empty, set all attributes from the dicitonary.
            model = CTGAN(
                log_frequency=parameter_set["log_frequency"],
                generator_lr=parameter_set["generator_lr"],
                generator_decay=parameter_set["generator_decay"],
                discriminator_lr=parameter_set["discriminator_lr"],
                discriminator_decay=parameter_set["discriminator_decay"],
                discriminator_steps=parameter_set["discriminator_steps"],
                verbose=False,
            )
        else:
            model = CTGAN()
            parameter_set = {"empty": "using default parameters"}

        # train the model on the given data set
        model.fit(df)
        # generate data equal to the size of the original data set
        new_data = model.sample(num_rows=df.shape[0])
        # evaluate the quality of the new data compared to the old data
        score = evaluate(
            new_data,
            df,
            metrics=[
                "CSTest",
                "KSTest",
                "BNLogLikelihood",
                "GMLogLikelihood",
                "LogisticDetection",
                "SVCDetection",
            ],
            aggregate=True,
        )
        elapsed_time = time() - start_time
        print("Score:", score)
        print(f"Elapsed time: {round(elapsed_time/60, 2)} minutes.")
        print("-------------------------------------\n")
        # save the best score and parameter set
        if score > best_score:
            best_parameter_set = parameter_set
            best_score = score
        count += 1
    # print out best parameters
    print("Best Parameters:")
    print("-------------------------------------")
    for x in best_parameter_set:
        print(x, ": ", best_parameter_set[x])
    print("-------------------------------------")
    print("Best Score:", best_score)
    print("-------------------------------------\n")

    # return best parameters
    return best_parameter_set

In [21]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
from pandas.core.common import SettingWithCopyWarning

simplefilter("ignore", category=ConvergenceWarning)
simplefilter("ignore", category=SettingWithCopyWarning)

N_ITER = 150

best_parameters = {}
RandomSearch(acute_stress, params=param_dist, n_iter=N_ITER)

Parameter Search for "      HR_mean  HRV_RMSSD  HRV_MeanNN  HRV_LFHF  RSP_mean  RRV_RMSSD  \
0    0.000000   1.000000    1.000000  0.224037  0.619060   0.635687   
1    0.115273   0.187134    0.867799  0.256498  0.755371   0.134214   
2    0.048016   0.344208    0.945329  0.075007  0.247844   0.107245   
3    0.056035   0.389607    0.934639  0.000000  0.000000   0.063393   
4    0.105911   0.117034    0.879118  0.067153  0.400969   0.000000   
..        ...        ...         ...       ...       ...        ...   
166  0.000000   0.582986    1.000000  0.094339  0.000000   0.000000   
167  0.190848   1.000000    0.791528  0.000000  0.036714   0.381408   
168  0.197764   0.509840    0.783975  0.376311  0.080187   0.923763   
169  1.000000   0.017698    0.000000  1.000000  0.564496   1.000000   
170  0.332384   0.474417    0.642975  0.924948  1.000000   0.505177   

     RRV_MeanBB  label  ID  
0      0.263531      1   4  
1      0.212412      1   4  
2      0.663463      1   4  
3      1.

Score: 0.2902059012591396
Elapsed time: 0.33 minutes.
-------------------------------------

Parameter Search for "      HR_mean  HRV_RMSSD  HRV_MeanNN  HRV_LFHF  RSP_mean  RRV_RMSSD  \
0    0.000000   1.000000    1.000000  0.224037  0.619060   0.635687   
1    0.115273   0.187134    0.867799  0.256498  0.755371   0.134214   
2    0.048016   0.344208    0.945329  0.075007  0.247844   0.107245   
3    0.056035   0.389607    0.934639  0.000000  0.000000   0.063393   
4    0.105911   0.117034    0.879118  0.067153  0.400969   0.000000   
..        ...        ...         ...       ...       ...        ...   
166  0.000000   0.582986    1.000000  0.094339  0.000000   0.000000   
167  0.190848   1.000000    0.791528  0.000000  0.036714   0.381408   
168  0.197764   0.509840    0.783975  0.376311  0.080187   0.923763   
169  1.000000   0.017698    0.000000  1.000000  0.564496   1.000000   
170  0.332384   0.474417    0.642975  0.924948  1.000000   0.505177   

     RRV_MeanBB  label  ID  
0  

Score: 0.4494367398709336
Elapsed time: 2.15 minutes.
-------------------------------------

Parameter Search for "      HR_mean  HRV_RMSSD  HRV_MeanNN  HRV_LFHF  RSP_mean  RRV_RMSSD  \
0    0.000000   1.000000    1.000000  0.224037  0.619060   0.635687   
1    0.115273   0.187134    0.867799  0.256498  0.755371   0.134214   
2    0.048016   0.344208    0.945329  0.075007  0.247844   0.107245   
3    0.056035   0.389607    0.934639  0.000000  0.000000   0.063393   
4    0.105911   0.117034    0.879118  0.067153  0.400969   0.000000   
..        ...        ...         ...       ...       ...        ...   
166  0.000000   0.582986    1.000000  0.094339  0.000000   0.000000   
167  0.190848   1.000000    0.791528  0.000000  0.036714   0.381408   
168  0.197764   0.509840    0.783975  0.376311  0.080187   0.923763   
169  1.000000   0.017698    0.000000  1.000000  0.564496   1.000000   
170  0.332384   0.474417    0.642975  0.924948  1.000000   0.505177   

     RRV_MeanBB  label  ID  
0  

Score: 0.20015349371978483
Elapsed time: 0.34 minutes.
-------------------------------------

Parameter Search for "      HR_mean  HRV_RMSSD  HRV_MeanNN  HRV_LFHF  RSP_mean  RRV_RMSSD  \
0    0.000000   1.000000    1.000000  0.224037  0.619060   0.635687   
1    0.115273   0.187134    0.867799  0.256498  0.755371   0.134214   
2    0.048016   0.344208    0.945329  0.075007  0.247844   0.107245   
3    0.056035   0.389607    0.934639  0.000000  0.000000   0.063393   
4    0.105911   0.117034    0.879118  0.067153  0.400969   0.000000   
..        ...        ...         ...       ...       ...        ...   
166  0.000000   0.582986    1.000000  0.094339  0.000000   0.000000   
167  0.190848   1.000000    0.791528  0.000000  0.036714   0.381408   
168  0.197764   0.509840    0.783975  0.376311  0.080187   0.923763   
169  1.000000   0.017698    0.000000  1.000000  0.564496   1.000000   
170  0.332384   0.474417    0.642975  0.924948  1.000000   0.505177   

     RRV_MeanBB  label  ID  
0 

Score: 0.4886411437668551
Elapsed time: 6.92 minutes.
-------------------------------------

Parameter Search for "      HR_mean  HRV_RMSSD  HRV_MeanNN  HRV_LFHF  RSP_mean  RRV_RMSSD  \
0    0.000000   1.000000    1.000000  0.224037  0.619060   0.635687   
1    0.115273   0.187134    0.867799  0.256498  0.755371   0.134214   
2    0.048016   0.344208    0.945329  0.075007  0.247844   0.107245   
3    0.056035   0.389607    0.934639  0.000000  0.000000   0.063393   
4    0.105911   0.117034    0.879118  0.067153  0.400969   0.000000   
..        ...        ...         ...       ...       ...        ...   
166  0.000000   0.582986    1.000000  0.094339  0.000000   0.000000   
167  0.190848   1.000000    0.791528  0.000000  0.036714   0.381408   
168  0.197764   0.509840    0.783975  0.376311  0.080187   0.923763   
169  1.000000   0.017698    0.000000  1.000000  0.564496   1.000000   
170  0.332384   0.474417    0.642975  0.924948  1.000000   0.505177   

     RRV_MeanBB  label  ID  
0  

...

Score: 0.3797022689591489
Elapsed time: 0.33 minutes.
-------------------------------------

Parameter Search for "      HR_mean  HRV_RMSSD  HRV_MeanNN  HRV_LFHF  RSP_mean  RRV_RMSSD  \
0    0.000000   1.000000    1.000000  0.224037  0.619060   0.635687   
1    0.115273   0.187134    0.867799  0.256498  0.755371   0.134214   
2    0.048016   0.344208    0.945329  0.075007  0.247844   0.107245   
3    0.056035   0.389607    0.934639  0.000000  0.000000   0.063393   
4    0.105911   0.117034    0.879118  0.067153  0.400969   0.000000   
..        ...        ...         ...       ...       ...        ...   
166  0.000000   0.582986    1.000000  0.094339  0.000000   0.000000   
167  0.190848   1.000000    0.791528  0.000000  0.036714   0.381408   
168  0.197764   0.509840    0.783975  0.376311  0.080187   0.923763   
169  1.000000   0.017698    0.000000  1.000000  0.564496   1.000000   
170  0.332384   0.474417    0.642975  0.924948  1.000000   0.505177   

     RRV_MeanBB  label  ID  
0  

Score: 0.3464746436401779
Elapsed time: 1.19 minutes.
-------------------------------------

Parameter Search for "      HR_mean  HRV_RMSSD  HRV_MeanNN  HRV_LFHF  RSP_mean  RRV_RMSSD  \
0    0.000000   1.000000    1.000000  0.224037  0.619060   0.635687   
1    0.115273   0.187134    0.867799  0.256498  0.755371   0.134214   
2    0.048016   0.344208    0.945329  0.075007  0.247844   0.107245   
3    0.056035   0.389607    0.934639  0.000000  0.000000   0.063393   
4    0.105911   0.117034    0.879118  0.067153  0.400969   0.000000   
..        ...        ...         ...       ...       ...        ...   
166  0.000000   0.582986    1.000000  0.094339  0.000000   0.000000   
167  0.190848   1.000000    0.791528  0.000000  0.036714   0.381408   
168  0.197764   0.509840    0.783975  0.376311  0.080187   0.923763   
169  1.000000   0.017698    0.000000  1.000000  0.564496   1.000000   
170  0.332384   0.474417    0.642975  0.924948  1.000000   0.505177   

     RRV_MeanBB  label  ID  
0  

Score: 0.45028955553875416
Elapsed time: 0.33 minutes.
-------------------------------------

Parameter Search for "      HR_mean  HRV_RMSSD  HRV_MeanNN  HRV_LFHF  RSP_mean  RRV_RMSSD  \
0    0.000000   1.000000    1.000000  0.224037  0.619060   0.635687   
1    0.115273   0.187134    0.867799  0.256498  0.755371   0.134214   
2    0.048016   0.344208    0.945329  0.075007  0.247844   0.107245   
3    0.056035   0.389607    0.934639  0.000000  0.000000   0.063393   
4    0.105911   0.117034    0.879118  0.067153  0.400969   0.000000   
..        ...        ...         ...       ...       ...        ...   
166  0.000000   0.582986    1.000000  0.094339  0.000000   0.000000   
167  0.190848   1.000000    0.791528  0.000000  0.036714   0.381408   
168  0.197764   0.509840    0.783975  0.376311  0.080187   0.923763   
169  1.000000   0.017698    0.000000  1.000000  0.564496   1.000000   
170  0.332384   0.474417    0.642975  0.924948  1.000000   0.505177   

     RRV_MeanBB  label  ID  
0 

Score: 0.378342314188559
Elapsed time: 0.32 minutes.
-------------------------------------

Best Parameters:
-------------------------------------
discriminator_decay :  0.0002980602398777751
discriminator_lr :  0.0017901333247018088
discriminator_steps :  10
generator_decay :  0.0004416387260647504
generator_lr :  0.00045743418650565575
log_frequency :  False
-------------------------------------
Best Score: 0.5790028651484477
-------------------------------------



{'discriminator_decay': 0.0002980602398777751,
 'discriminator_lr': 0.0017901333247018088,
 'discriminator_steps': 10,
 'generator_decay': 0.0004416387260647504,
 'generator_lr': 0.00045743418650565575,
 'log_frequency': False}

In [36]:
best_parameters

{}